In [ ]:
import torch
import torch.nn as nn
import numpy as np

from einops import rearrange

from mosaic_sdf import MosaicSDF
from shape_sampler import ShapeSampler
from optimizer import MosaicSDFOptimizer
from mosaic_sdf_visualizer import MosaicSDFVisualizer

from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

In [ ]:
cube_mesh_path = 'data/cube.obj'
teapot_mesh_path = 'data/utah_teapot.obj'
cow_mesh_path = 'data/cow_mesh/cow.obj'

cube_wireframe_path = 'data/cube_wireframe.obj'

sdf_shape_path = teapot_mesh_path

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
from ray import tune
import os


config = {   
    'device': device,
    # 'shape_sampler': shape_sampler,  # Adjust accordingly
    'shape_path': os.path.abspath(sdf_shape_path),  # Adjust accordingly
    
    # mosaicSDF params
    'grid_resolution': 7,
    # 'n_grids': 1024,
    # 'n_grids': 128,
    # 'n_grids': 8,
    'n_grids': tune.grid_search([ 512 ]),
    'points_random_spread': tune.grid_search([.01, .03, .05]),

    'val_points_random_spread': .03,
    # 'mosaic_scale_multiplier': 3,
    'mosaic_scale_multiplier': tune.grid_search([ 2, 3, 5 ]),
    
    # optimizer params
    # 'lr': 1e-4,
    'lr': tune.grid_search([ 1e-3, 1e-4]),
    'weight_decay': 0.0,
    "b1": tune.grid_search([0.5, 0.9]),
    "b2": .999,

    # lerp between l1 and l2 losses
    'lambda_val': .1,
    
    
    # optimization params
    # 'n_epochs':  tune.choice([1, 4, 8, 16]),
    # 'n_epochs':  tune.grid([ 8 ]),
    
    'n_epochs':  2,

    'points_in_epoch': 4096,
    'points_sample_size': 32,
    # 'gradient_accumulation_steps': 1,
    'gradient_accumulation_steps': tune.grid_search([1, 4, 8]),

    'eval_every_nth_points': 512,
    'val_size': 2048,
    'points_sample_size_eval_scaler': 4, # can sample faster during eval

    'project_name': 'mosaicSDF_tea_pot.03',
    'log_to_wandb': True, 
    'log_to_console': False,
    
    # other debug stuff
    'output_graph': False,
    'points_random_sampling': False
}


In [ ]:
from ray.train import RunConfig, CheckpointConfig
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.bayesopt import BayesOptSearch
from ray.data import DataContext
from ray.train import Checkpoint

# from ray.air.integrations.mlflow import MLflowLoggerCallback
# mlflow_tracking_uri = "http://localhost:5000"

driver_ctx = DataContext.get_current()

trainable_with_resources = tune.with_resources(MosaicSDFOptimizer, {"cpu": 10, "gpu": 1})

wb_exclude = ['training_iteration',
              'timestamp',
              'time_since_restore',
              'iterations_since_restore']

tune_storage_path = f"./out/tune/{config['project_name']}"
tune_storage_path = os.path.abspath(tune_storage_path)

# experiment_name = 'mosaic_sdf_smoke'

# print(f"experiment_name: {experiment_name}")

tuner = tune.Tuner(
    trainable_with_resources,
    run_config=RunConfig(
        storage_path=tune_storage_path, 
        name=config['project_name'],
        stop={"training_iteration": config['n_epochs']},
        callbacks= [
         #   WandbLoggerCallback(project="hello_tune", excludes=wb_exclude),
            # MLflowLoggerCallback(
            #         tracking_uri=mlflow_tracking_uri,
            #         experiment_name=experiment_name,
            #         save_artifact=False,
            #     )
         ],
        checkpoint_config=CheckpointConfig(
            num_to_keep=10,
            checkpoint_frequency=0,  #checkpoint disabled!
            checkpoint_at_end=True
        )),
    
    tune_config=tune.TuneConfig(
        # num_samples=1,
        # metric="test_loss_pixel", 
        # mode="min",
        max_concurrent_trials=1,
        # search_alg=algo,
        scheduler=ASHAScheduler(metric='val_loss', mode="min", grace_period=10),
    ),
    # param_space=train_config if continue_tune else tuning_config,
    param_space=config
)


continue_tune = False
if not continue_tune:
    results = tuner.fit()
else:
    print('resuming')
    restored_tuner = tune.Tuner.restore(
        Checkpoint.from_directory(os.path.join(tune_storage_path, config['project_name'])).path,
        trainable=trainable_with_resources,
        # Re-specify the `param_space` to update the object references.
        param_space=config,
        resume_unfinished=True,
    )



In [ ]:
optimizer = None # load optimizer

In [ ]:

visualizer = MosaicSDFVisualizer(optimizer.model, optimizer.shape_sampler, 
    device, template_mesh_path=cube_wireframe_path)#, requires_grad=False)

shape_sampler = ShapeSampler.from_file(sdf_shape_path, device='cuda')

In [ ]:
from utils_vis import compare_shapes

compare_shapes(shape_sampler, visualizer, 32, show_mosaic_grids=False,
               show_gt_mesh=True, show_gt_sdf=False, show_mosaic_sdf=False)


In [ ]:
compare_shapes(32, show_mosaic_grids=False)

In [ ]:

# Assuming you're not using a data loader for training
optimizer.train()  # Pass None or adjust `train` method to not require `train_loader`



# I am also not sure autograd is what we need, how it works in this case at all?

# # To save the optimized model
# optimizer.save_checkpoint('path/to/save/checkpoint')

# # To load an existing model
# optimizer.load_checkpoint('path/to/existing/checkpoint.pth')


In [ ]:
compare_shapes(32)

In [ ]:
# if True:
#     with torch.no_grad():
#         meshes = visualizer.create_state_meshes(
#             show_mosaic_grids=False,
#             show_target_mesh=False,
#             resolution=16
#             )
        
#         # Render the plotly figure
#         fig = plot_scene({
#             "subplot1": {
#                 "mesh": meshes
#             }
#         })
#         fig.show()    
    


In [ ]:
# with torch.no_grad():
#     resolution = 8
#     grid_points = torch.stack(torch.meshgrid(
#             torch.linspace(-1, 1, resolution),
#             torch.linspace(-1, 1, resolution),
#             torch.linspace(-1, 1, resolution)
#         ), dim=-1).reshape(-1, 3)#.to(device)

#     sdf_values = optimizer.model(grid_points.to(device))

In [ ]:
sdf_meshes._verts_list

In [ ]:
import torch
import numpy as np
from skimage.measure import marching_cubes
from pytorch3d.structures import Meshes
from pytorch3d.io import save_obj
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader
)
mosaic_sdf = optimizer.model
resolution = 16

# Assuming 'mosaic_sdf' is your MosaicSDF instance and 'resolution' is the desired grid resolution
grid_points = torch.stack(torch.meshgrid(
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution)
), dim=-1).reshape(-1, 3).to(config['device'])

# Get the SDF values at these points
sdf_values = mosaic_sdf(grid_points).detach().cpu().numpy()
sdf_volume = sdf_values.reshape(resolution, resolution, resolution)
sdf_volume.shape

In [ ]:
grid_points.shape

In [ ]:

# Run marching cubes to get vertices, faces, and normals
verts, faces, normals, _ = marching_cubes(sdf_volume, level=0)
faces = faces + 1  # skimage has 0-indexed faces, while PyTorch3D expects 1-indexed

# Convert to PyTorch tensors
verts = torch.tensor(verts, dtype=torch.float32)
faces = torch.tensor(faces, dtype=torch.int64)

# Create a PyTorch3D mesh
mesh = Meshes(verts=[verts], faces=[faces])

# Initialize a renderer
R, T = look_at_view_transform(2.7, 0, 90)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
raster_settings = RasterizationSettings(image_size=512)
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=SoftPhongShader(device=device, cameras=cameras, lights=lights)
)

# Render the mesh
images = renderer(mesh)


In [ ]:
num_check_x = torch.rand((3, 3))
forward = lambda x: torch.sum(x ** 2, axis=1)

num_grad = optimizer.compute_gradient_numerically(num_check_x, forward)

In [ ]:
num_grad

In [ ]:
num_check_x * 2

In [ ]:

# Define the quadratic function
def quadratic_function(points):
    return torch.sum(points ** 2, dim=-1)

# Known analytical gradients for the quadratic function
def analytical_gradients(points):
    return 2 * points

# Define points for which to compute gradients
points = torch.tensor([[1.0, 2.0, 3.0],
                       [4.0, 5.0, 6.0],
                       [7.0, 8.0, 9.0]], requires_grad=True)

# Compute the function output
function_output = quadratic_function(points)

# Compute analytical gradients
true_gradients = analytical_gradients(points)

# Compute gradients using autograd
function_output.backward(torch.ones_like(function_output))
autograd_gradients = points.grad

# Compute gradients using the numerical method
numerical_gradients = optimizer.compute_gradient_numerically(points, quadratic_function, delta=1e-2)

# Compare numerical gradients to true gradients
gradient_difference = torch.abs(numerical_gradients - true_gradients)

print("True gradients:\n", true_gradients)
print("Autograd gradients:\n", autograd_gradients)
print("Numerical gradients:\n", numerical_gradients)
print("Gradient difference:\n", gradient_difference)

# Check if the numerical gradients are close to the true gradients
assert torch.allclose(numerical_gradients, true_gradients, atol=1e-3), "Numerical gradients do not match true gradients closely enough."


In [ ]:
## test vars

volume_centers = torch.tensor([
    [0.1, 0, 0.5], 
    [0.5, 0.2, 0], 
    [0.7, 0.1, 0.6], 
    [1, 1, .5], 
])

scales = torch.tensor([1, 1, 1, .4])

points = torch.tensor([
    [0.5, 0.5, 0], 
    [1, 0, .5], 
])

k = 7

sdf_values = torch.rand((scales.shape[0], k, k ,k))


In [ ]:
# # Using the updated ShapeSampler class
# shape_sampler = ShapeSampler("sphere")
# center = torch.tensor([0.0, 0.0, 0.0])
# scale = 2.0

# # Compute local SDF values
# local_sdf_values = compute_local_sdf(shape_sampler, center, scale, grid_resolution=3)
# print(local_sdf_values.shape) 